### FREE SLIME

no comment scrapebook

In [11]:
import spotipy
import pandas as pd
import requests
import numpy as np
import time
from requests.exceptions import ReadTimeout
from spotipy.exceptions import SpotifyException
from timeit import default_timer as timer
from datetime import timedelta
import configparser
from spotipy.oauth2 import SpotifyClientCredentials
from pandas.api.types import CategoricalDtype
import configparser
import os

In [12]:
current_dir = os.path.dirname(os.path.abspath("Young Thug"))

config_path = os.path.join(current_dir, '..', 'config.ini')

config = configparser.ConfigParser()

config.read(config_path)

client_id = config.get('credentials', 'Client_ID')
client_secret = config.get('credentials', 'Client_Secret')

client_credentials_manager = SpotifyClientCredentials(
                                client_id = client_id, client_secret = client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

In [13]:
def timekeeper(seconds):
    minutes = seconds // 60
    seconds = seconds % 60
    if minutes > 0:
        return f"{minutes} minutes, {seconds} seconds"
    else:
        return f"{seconds} seconds"

In [5]:
def collect_spotify_data():
    start_time = timer()
    data = []

    for i in range(0, 1000, 50):
        try:
            track_results = sp.search(q='artist: Young Thug', type='track', limit=50, offset=i)
            for item in track_results['tracks']['items']:
                track_info = {
                    'artist_name': item['artists'][0]['name'],
                    'track_name': item['name'],
                    'track_id': item['id'],
                    'album_name': item['album']['name'],
                    'album_id': item['album']['id'],
                    'release_date': item['album']['release_date'],
                    'popularity': item['popularity'],
                    'explicit': item['explicit']
                }
                
                audio_features = sp.audio_features(item['id'])[0]
                if audio_features is not None:
                    audio_info = {
                        'danceability': audio_features.get('danceability', float('nan')),
                        'duration_ms': audio_features.get('duration_ms', float('nan')),
                        'energy': audio_features.get('energy', float('nan')),
                        'key': audio_features.get('key', float('nan')),
                        'loudness': audio_features.get('loudness', float('nan')),
                        'mode': audio_features.get('mode', float('nan')),
                        'speechiness': audio_features.get('speechiness', float('nan')),
                        'acousticness': audio_features.get('acousticness', float('nan')),
                        'instrumentalness': audio_features.get('instrumentalness', float('nan')),
                        'liveness': audio_features.get('liveness', float('nan')),
                        'valence': audio_features.get('valence', float('nan')),
                        'tempo': audio_features.get('tempo', float('nan')),
                        'time_signature': audio_features.get('time_signature', float('nan'))
                    }
                    track_info.update(audio_info)
                else:
                    audio_info = {
                        'danceability': float('nan'),
                        'duration_ms': float('nan'),
                        'energy': float('nan'),
                        'key': float('nan'),
                        'loudness': float('nan'),
                        'mode': float('nan'),
                        'speechiness': float('nan'),
                        'acousticness': float('nan'),
                        'instrumentalness': float('nan'),
                        'liveness': float('nan'),
                        'valence': float('nan'),
                        'tempo': float('nan'),
                        'time_signature': float('nan')
                    }
                    track_info.update(audio_info)

                
                if len(item['artists']) > 1:
                    featured_artists = [artist['name'] for artist in item['artists'][1:]]
                    track_info['featured_artists'] = featured_artists
                else:
                    track_info['featured_artists'] = []

                data.append(track_info)

        except requests.exceptions.ReadTimeout as e:
            print(f"Timeout error: {e}. Retrying in 5 seconds...")
            time.sleep(5) 
        except requests.exceptions.RequestException as e:
            print(f"API error: {e}")
            break 

    df_raw = pd.DataFrame(data)
    end_time = timer()
    elapsed_time = int(end_time - start_time)
    print(f"Elapsed time:", timekeeper(elapsed_time))
    return df_raw

df_spotify = collect_spotify_data()


Timeout error: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5). Retrying in 5 seconds...
Elapsed time: 3 minutes, 22 seconds


In [8]:
thug = df_spotify[df_spotify['artist_name']== 'Young Thug']
thug.to_csv('raw material.csv')

In [19]:
thug = pd.read_csv('raw material.csv')
thug.drop(["Unnamed: 0"], axis=1, inplace= True)
thug

,artist_name,track_name,track_id,album_name,album_id,release_date,popularity,explicit,danceability,duration_ms,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
0,Young Thug,Check,1jk9c0gZI0wbn5GvAj7PBV,Barter 6,0BsMZIueWsJLWng8A7sE8e,2015-04-16,63,True,0.667,230693,...,-5.164,0,0.3230,0.042800,0.000000,0.1830,0.368,110.953,4,[]
1,Young Thug,Hot (feat. Gunna),5Z8HZM6iQMhhqyPcCGY5g9,So Much Fun,1bnHPO4dKK7IjvgrtVBcQh,2019-08-16,65,True,0.855,193027,...,-8.044,0,0.2350,0.063000,0.000000,0.0772,0.559,111.997,4,['Gunna']
2,Young Thug,Oh U Went (feat. Drake),7qLr3HMApUbyDkUvgIvHnB,BUSINESS IS BUSINESS,6qElaeK7SHsEzb8uV8eG3D,2023-06-23,65,True,0.808,185333,...,-5.880,0,0.2360,0.051700,0.000000,0.0819,0.465,136.010,4,['Drake']
3,Young Thug,Digits,4cg1yakyRSIOjxKM2I7J1q,Slime Season 3,2z4c8M8aVzl7CTobIp36KF,2016-03-25,66,True,0.769,176387,...,-4.228,0,0.0537,0.032500,0.000000,0.1160,0.294,131.932,4,[]
4,Young Thug,Money On The Dresser,0k88aN0xmk8Xn1zdnyTiPG,BUSINESS IS BUSINESS,6qElaeK7SHsEzb8uV8eG3D,2023-06-23,46,True,0.896,133000,...,-6.903,1,0.2670,0.000107,0.008260,0.1050,0.441,159.982,4,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,Young Thug,Hot (feat. Gunna),4HKumMuXXk8BZG2Pb8OCPo,Gym Workout Core Build 2024,3hTfRPAdByjYJyu2fO5ZUe,2024-01-12,0,True,0.855,193027,...,-8.044,0,0.2340,0.063600,0.000000,0.0772,0.559,111.992,4,['Gunna']
457,Young Thug,Oh U Went (feat. Drake),6phLw7zHlraCag28y1wYz1,it gets hot in the kitchen,4aNFzGht2zlJwbkwFsc4eV,2024-05-16,0,True,0.808,185333,...,-5.880,0,0.2360,0.051700,0.000000,0.0819,0.465,136.004,4,['Drake']
458,Young Thug,Oh U Went (feat. Drake),2pXX6Bwxxhim77ZkOm6ZSg,Rap & Run,43z8p0EiMA6I76SMRPz7tS,2023-09-15,0,True,0.808,185333,...,-5.880,0,0.2360,0.051700,0.000000,0.0819,0.465,136.010,4,['Drake']
459,Young Thug,Parade on Cleveland,0xbDfBA9IthHfv1YmD9M4R,It's cold and i'm sad,1GRZAteRVU6DiK4SoLi9Lg,2024-01-19,0,True,0.526,228267,...,-13.127,0,0.5660,0.447000,0.000005,0.1110,0.354,165.991,1,['Drake']


In [40]:
thug['album_name'].unique()

array(['Barter 6', 'So Much Fun', 'BUSINESS IS BUSINESS',
       'Slime Season 3', 'Punk', 'Slime Season 4', 'So Much Fun (Deluxe)',
       'From A Man', 'JEFFERY', 'Gangster Shit', 'Stoner', 'Dope Boy',
       "BUSINESS IS BUSINESS (Metro's Version)", "I'm Up", 'Tick Tock',
       'On the Run', 'SUPER SLIMEY', 'Beautiful Thugger Girls',
       'Rich Homie Cartel Vol 1', 'Hear No Evil', 'Slime Language',
       'Lost Files', 'Young Martha', 'Tell Her Nothing',
       'In Trap We Trust, Vol. 3', 'I Came from Nothing 2',
       'Homie (feat. Meek Mill)', 'Liger', '1017 Thug',
       "What's Poppin - Fresh Rap", "Ballin' & Stuntin', Vol. 11",
       'Brick Sqaud', 'Up to Something', 'Anybody (feat. Nicki Minaj)',
       'Trap', 'Pacifier', 'Rich Homie Gang - Hood Lifestyle',
       'Gunna: A Gift & A Curse Setlist',
       'Before Anythang (Original Motion Picture Soundtrack)',
       '1017 Thug 3 The Finale', 'Trappin Aint Eazy Vol 1',
       'D.A.B. Muzic, Vol. 1', 'Pop Music for Headph

<p align="center "# FREE SLIME>

In [ ]:
studio_albums = [
    "BUSINESS IS BUSINESS",
    "So Much Fun",
    'Punk'
]

collaboration_albums = [
    "SUPER SLIMEY",
    "Slime & B",
    "MigoThuggin"
]

deluxe_albums = [
    "So Much Fun (Deluxe)",
    "BUSINESS IS BUSINESS (Metro's Version)"          

]

ep = [
    "On the Run",
    "Young Martha",
    "Hear No Evil"
]

compilation_albums = [
    "Slime Language 2",
    "Slime Language",
]

mixtapes = [
    'I Came from Nothing 2',
    "1017 Thug",
    "The Purple Album"
    "Street Is Watching (The Mix-Tape)",
    "RG2",
    "Young Thugga Mane La Flare",
    "Ready for War",
    "Brick Squad"
    "1017 Thug 3 The Finale"
]

commercial_mixtapes = [
    'Barter 6', 
    'Slime Season 3', 
    'Slime Season 4',  
    'Beautiful Thugger Girls', 
    'JEFFERY', 
    "I'm Up", 
]